In [2]:
# -*- coding: utf-8 -*-
# pip install geopandas libpysal esda shapely pyproj

import os
import numpy as np
import pandas as pd
import geopandas as gpd
from libpysal.weights import KNN
from esda.getisord import G_Local

# ==============================
# 0) 설정
# ==============================
INPUT_PATH = "violations.json"   
CRS_WGS84  = "EPSG:4326"
CRS_KM     = "EPSG:5186"     # (한국) 미터 좌표로 변환용
PERM       = 499             # Gi* 랜덤화 반복(속도/정확도 균형)
K_NEI      = 8               # KNN 이웃 수(6~12 권장)

# 핫스팟 조건 (간단/직관)
P_THRESH        = 0.15        # p < 0.10
Z_THRESH        = 1.28        # Z > 1.65 (≈90%)
CENTER_COUNT_Q  = 0.50        # 중심 count 하한: 상위 40% (분위수 0.60)
LOCAL_SUM_Q     = 0.60        # 주변 합 하한: 상위 25% (분위수 0.75)

# 시간 구간
PERIODS = {
    "morning":   {"label":"오전",  "hours": list(range(6,11))},
    "lunch":     {"label":"점심",  "hours": list(range(11,15))},
    "afternoon": {"label":"오후",  "hours": list(range(15,18))},
    "evening":   {"label":"저녁",  "hours": list(range(18,23))},
}

# 시각화 반지름(간단 공식)
def radius_from(local_sum, z):
    r = 6 + 0.40*np.sqrt(max(local_sum, 0)) + 1.6*max(0.0, z)
    return float(min(40, max(6, r)))

# ==============================
# 1) 파일 읽기
# ==============================
if not os.path.exists(INPUT_PATH):
    raise SystemExit(f"파일이 없습니다: {INPUT_PATH}")

ext = os.path.splitext(INPUT_PATH)[1].lower()
if ext == ".json":
    df = pd.read_json(INPUT_PATH)
elif ext == ".csv":
    df = pd.read_csv(INPUT_PATH)
else:
    raise SystemExit("지원 확장자: .json / .csv")

required = {"lat","lon","hour","count"}
assert required.issubset(df.columns), f"다음 컬럼 필요: {required}"

# ==============================
# 2) 전역 지점 세트 & 전역 KNN W (시간대마다 동일)
# ==============================
sites = df[["lat","lon"]].drop_duplicates().reset_index(drop=True)
if len(sites) < 5:
    raise SystemExit("유효 지점이 너무 적습니다(≥5).")

g_sites = gpd.GeoDataFrame(
    sites.copy(),
    geometry=gpd.points_from_xy(sites.lon, sites.lat),
    crs=CRS_WGS84
).to_crs(CRS_KM)

coords = np.column_stack([g_sites.geometry.x.values, g_sites.geometry.y.values])
W = KNN.from_array(coords, k=K_NEI)
W.transform = "R"  # 행표준화

# 전역 인덱스 매핑
site_index = { (row.lat, row.lon): i for i, row in sites.iterrows() }
N = len(sites)

# ==============================
# 3) 시간대별 핫스팟 계산
# ==============================
period_meta = []

for key, meta in PERIODS.items():
    hrs = meta["hours"]

    # (2-1) 해당 구간 데이터 → 좌표별 합산
    sub = df[df["hour"].isin(hrs)].copy()
    sub_agg = sub.groupby(["lat","lon"], as_index=False)["count"].sum()

    # (2-2) 전역 순서에 맞춘 y 벡터(결측=0)
    y = np.zeros(N, dtype=float)
    for _, r in sub_agg.iterrows():
        idx = site_index.get((r["lat"], r["lon"]))
        if idx is not None:
            y[idx] = float(r["count"])

    # (3-1) Gi*
    gi = G_Local(y, W, permutations=PERM)

    # (3-2) 로컬합/이웃수 (자기+KNN)
    neighbors = W.neighbors
    local_sum = np.zeros(N, dtype=float)
    local_n   = np.zeros(N, dtype=int)
    for i in range(N):
        nbs = neighbors.get(i, [])
        idxs = [i] + nbs
        local_sum[i] = float(np.sum(y[idxs]))
        local_n[i]   = len(nbs)

    # (3-3) 결과 프레임
    out = g_sites.to_crs(CRS_WGS84).copy()
    out.loc[:, "GiZ"]          = gi.Zs
    out.loc[:, "GiP"]          = gi.p_sim
    out.loc[:, "local_sum"]    = local_sum
    out.loc[:, "local_n"]      = local_n
    out.loc[:, "center_count"] = y

    # (3-4) 단순 필터: "중심도 크고, 주변도 크고, 뜨거움(양의 유의 Gi*)"
    base = out  # 전 지점 기준으로 분위수 계산(안정적)
    cc_cut = float(np.nanquantile(base["center_count"], CENTER_COUNT_Q))
    ls_cut = float(np.nanquantile(base["local_sum"],     LOCAL_SUM_Q))

    hot = out[
        (out["GiP"] < P_THRESH) &
        (out["GiZ"] > Z_THRESH) &
        (out["center_count"] >= cc_cut) &
        (out["local_sum"]    >= ls_cut)
    ].copy()

    # 시각화 속성
    hot.loc[:, "color"]  = "#e53935"
    hot.loc[:, "radius"] = [radius_from(s, z) for s, z in zip(hot["local_sum"], hot["GiZ"])]
    hot.loc[:, "period_key"]   = key
    hot.loc[:, "period_label"] = meta["label"]

    # 내보내기(JSON; 카카오맵 그대로 사용 가능)
    export = hot[["lat","lon","period_key","period_label",
                  "center_count","local_sum","local_n","GiZ","GiP","color","radius"]].copy()
    export.rename(columns={"center_count":"count"}, inplace=True)
    export.to_json(f"hotspots_period_{key}_hot.json", orient="records", force_ascii=False)

    period_meta.append({"key": key, "label": meta["label"], "n": int(len(export))})
    print(f"[{meta['label']}] 저장: hotspots_period_{key}_hot.json (핫스팟 {len(export)}개)")

# 메타
pd.DataFrame(period_meta).to_json("available_periods_hot.json", orient="records", force_ascii=False)
print("완료: available_periods_hot.json 및 각 구간 hot JSON 생성")


c:\Users\wonny\anaconda3\Lib\site-packages\libpysal\weights\distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


[오전] 저장: hotspots_period_morning_hot.json (핫스팟 123개)
[점심] 저장: hotspots_period_lunch_hot.json (핫스팟 236개)
[오후] 저장: hotspots_period_afternoon_hot.json (핫스팟 149개)
[저녁] 저장: hotspots_period_evening_hot.json (핫스팟 110개)
완료: available_periods_hot.json 및 각 구간 hot JSON 생성
